In [2]:
# 数据处理
import pandas as pd
import numpy as np
import json
import csv
import random
import math
from itertools import product
from community import community_louvain



# 网络分析与可视化
import networkx as nx
from networkx.readwrite import json_graph
from pyvis import network as net
import matplotlib.pyplot as plt
import powerlaw # Power laws are probability distributions with the form:p(x)∝x−α

##  构建测试系统用的Master Concept Network

In [3]:
with open('02_Graph/ConceptNetwork_Complete_CCFD-Similarity-Matrix.json') as f:
    js_graph = json.load(f)
    G_ConceptNetwork = json_graph.node_link_graph(js_graph)

In [ ]:
# # 测试Edge Filter的参数用。用完整的，未经过边筛选的 G_ConceptNetwork
# for u,v,e in G_ConceptNetwork.edges(data=True):
#     if ((e['weight'] < 0.66) & (e['weight']>0.65)):
#         shared_features = Extract_Associate_SemanticFeature(df_complete, 'Concept', 'Feature', u, v)
#         if shared_features !=0:
#             print(e['weight'],u,v,shared_features)

In [4]:
def filter_edge(u,v):
    if G_ConceptNetwork[u][v]['weight'] > 0.52:
    # if G[u][v]['weight'] > 0:
        return G_ConceptNetwork[u][v]

G_Subgraph = nx.subgraph_view(G_ConceptNetwork,filter_edge=filter_edge,)
largest_cc = max(nx.connected_components(G_Subgraph), key=len)
G_MasterConceptNetwork = G_Subgraph.subgraph(largest_cc)


# # 将全图导出
# with open('02_Graph/ConceptNetwork_Complete_CCFD-Similarity-Matrix_Weight-0.52.json','w+') as f:
#     f.write(json.dumps(nx.node_link_data(G_ConceptNetwork), ensure_ascii=False))


# 将全图导出 for Gephi
# nx.write_gexf(G_MasterConceptNetwork, "02_Graph/ConceptNetwork_Complete_CCFD-Similarity-Matrix_Weight-0.52.gexf")

## Small-World Network Test

In [7]:
## To Do 很久

# nx.average_shortest_path_length(G_MasterConceptNetwork)

In [ ]:
# # Power law test

# degree_sequence = sorted([d for n, d in G_ConceptNetwork.degree()], reverse=True) # used for degree distribution and powerlaw test
# fit = powerlaw.Fit(degree_sequence) 
# fit.power_law.alpha
# fit.power_law.sigma
# fit.distribution_compare('power_law', 'exponential')
# # fig2 = fit.plot_pdf(color='b', linewidth=2)
# # fit.power_law.plot_pdf(color='g', linestyle='--', ax=fig2)


## 计算图指标 Graph Metrics

### degree / betweennes_centrality / closeness_centrality / laplacian_centrality / clustering coefficent / pagerank

In [64]:
# 计算 degree centrality 很快
degree_centrality = nx.degree_centrality(G_MasterConceptNetwork)

# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, degree_centrality, "degree_centrality")


In [70]:
# 计算 pagerank 很快
pagerank = nx.pagerank(G_MasterConceptNetwork, alpha=0.9, weight='weight')

# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, pagerank, "pagerank")

In [65]:
# 计算 closeness centrality 时间很长12min
closeness_centrality = nx.closeness_centrality(G_MasterConceptNetwork)

# 重映射值域
# closeness_centrality.update((x, translate(y,min(closeness_centrality.values()), max(closeness_centrality.values()), 0, 20 )) for x, y in closeness_centrality.items())

# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, closeness_centrality, "closeness_centrality")


In [66]:
# 计算 betweenness centrality 时间很长
# betweenness_centrality = nx.betweenness_centrality(G_MasterConceptNetwork)


# 写入节点属性（先用0代替）
nx.set_node_attributes(G_MasterConceptNetwork, 0, "betweenness_centrality")
# nx.set_node_attributes(G_MasterConceptNetwork, betweenness_centrality, "closeness_centrality")


In [80]:
# 计算 Laplacian centrality
# https://andrewpwheeler.com/2015/07/29/laplacian-centrality-in-networkx-python/

def lap_energy(graph, weight='weight'):
  # degrees = graph.degree(weight=weight)
  degrees = {node:val for (node, val) in graph.degree(weight=weight)}

  d1 = sum(v**2 for i,v in degrees.items())
  wl = 0
  for i in graph.edges(data=True):
    wl += (i[2].get(weight))**2
  return [d1,2*wl]

def cw(graph,node,weight='weight'):
  neis = graph.neighbors(node)
  ne = graph[node]
  cw,sub = 0,0
  for i in neis:
    we = ne[i].get(weight)
    od = graph.degree(i,weight=weight)
    sub += -od**2 + (od - we)**2
    cw += we**2
  return [cw,sub]

def lap_cent_weighted(graph, nbunch=None, norm=False, weight='weight'):
  if nbunch is None:
    vs = graph.nodes()
  else:
    vs = nbunch
  if norm == True:
    fe = lap_energy(graph,weight=weight)
    den = float(fe[0]+fe[1])
  else:
    den = 1
  result = {}
  for i in vs:
     d2 = graph.degree(i,weight=weight)
     w2 = cw(graph,i,weight=weight)
     fin = d2**2 - w2[1] + 2*w2[0]
     result[i]=fin/den
  return result

laplacian_centrality = lap_cent_weighted(G_MasterConceptNetwork, norm=True)


# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, laplacian_centrality, "laplacian_centrality")


In [63]:
# To Do
# 计算 clustering coefficient
clustering_coefficent = nx.clustering(G_MasterConceptNetwork, weight='weight')

# 写入节点属性（先用0代替）
nx.set_node_attributes(G_MasterConceptNetwork, clustering_coefficent, "clustering_coefficent")
# nx.set_node_attributes(G_MasterConceptNetwork, betweenness_centrality, "closeness_centrality")

In [87]:
# 计算 louvain community
louvain_community = community_louvain.best_partition(G_MasterConceptNetwork)

# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, louvain_community, "cluster")

In [102]:
G_MasterConceptNetwork.nodes['拇指']

{'subcategory': '身体部位',
 'maincategory': '身体部位',
 'closeness_centrality': 0.09451331778568799,
 'clustering_coefficent': 0.3978035506654546,
 'degree_centrality': 0.024675324675324677,
 'betweenness_centrality': 0,
 'pagerank': 0.0017751261682123973,
 'laplacian_centrality': 0.0011701986802920587,
 'louvain_community': 16}

## 在Gephi中计算布局

In [6]:
nx.write_gexf(G_MasterConceptNetwork, "02_Graph/MasterConceptNetwork.gexf")

------ Gephi Force Atalas 2 Layout / Export yo graphml ------

In [8]:
# From Gephi to G6 
G=nx.read_graphml('02_Graph/MasterConceptNetwork.graphml')

for node in G:
    del G.nodes[node]['r']
    del G.nodes[node]['g']
    del G.nodes[node]['b']
    del G.nodes[node]['size']
    del G.nodes[node]['label']

nx.node_link_data(G)

{'directed': False,
 'multigraph': False,
 'graph': {'node_default': {}, 'edge_default': {}},
 'nodes': [{'subcategory': '工具',
   'maincategory': '人造物',
   'closeness_centrality': 0.09340126152353227,
   'clustering_coefficent': 0.6113577,
   'degree_centrality': 0.01038961038961039,
   'betweenness_centrality': 0,
   'pagerank': 0.0005928941206861465,
   'laplacian_centrality': 0.0004178089757526054,
   'louvain_community': 0,
   'x': 673.2981,
   'y': 99.41043,
   'id': '安全带'},
  {'subcategory': '工具',
   'maincategory': '人造物',
   'closeness_centrality': 0.09433962264150944,
   'clustering_coefficent': 0.34177336,
   'degree_centrality': 0.014285714285714285,
   'betweenness_centrality': 0,
   'pagerank': 0.0007667517644426217,
   'laplacian_centrality': 0.0006127104423340437,
   'louvain_community': 0,
   'x': 665.06274,
   'y': 231.22034,
   'id': '安全帽'},
  {'subcategory': '酒水饮料',
   'maincategory': '食物',
   'closeness_centrality': 0.07670850767085077,
   'clustering_coefficent': 0.

In [9]:
# 导出 for G6
with open('02_Graph/MasterConceptNetwork_G6.json','w+') as f:
    f.write(json.dumps(nx.node_link_data(G), ensure_ascii=False).replace('links','edges'))

# 导出 for networkx
with open('02_Graph/MasterConceptNetwork.json','w+') as f:
    f.write(json.dumps(nx.node_link_data(G), ensure_ascii=False))